# Attractors and Oscillation
- running simulations
- analyzing data
- plotting

## setup

In [ ]:
import os
import sys
sys.path.insert(0, "..")
import numpy as np
import tqdm

import experiments_eif
import ExperimentAnalysis
import analysis
import persistence 
import utils
import mp
import attractor


In [ ]:
# adapt this to the desired path where the simulation is tb saved
# ensure sufficient disk space
base_path = "/media/ben/fbe4ea39-a83a-4758-9ccf-506225bbc38d/bla/"

## run simulations

### single simulations (from python)

In [ ]:
base_path_code = os.path.join(base_path, "code")
fname = "my_experiment.h5"
fnameu = "my_experiment_unweighted.h5"

In [ ]:
# parameters
simtime = 2250.0
rpe = 3.9
rpi = 5.15
esize = 4000
sparsity = 0.05 
stimuluspatternidx = 0
perturbation = 0.0
beta = 0.5
minusbeta = 1.0
continuousstim = False
weighted = True
norm = 4.25

In [ ]:
# generate patterns
pattern = experiments_eif.generate_fixed_patterns(
    esize=esize, sparsity=sparsity, numpatterns=20
)


In [ ]:
for sim in [fname, fnameu]:
    experiments_eif.run_exp_eif_attr_blocked_stimulus(simtime, os.path.join(base_path_code,sim), rpe, rpi, esize, sparsity, pattern, stimuluspatternidx, perturbation, beta, minusbeta, continuousstim, weighted, norm)

### batch simulation (multiprocessed - from cli)
- running directly from the cli continuously updates mp progress bar
- specifying parameters 
    - single value 'x'
    - a comma separated list 'x,y,z' 
    - slice '[0.0:1.01:0.02]' ~ (start,end,stop)

In [ ]:
# cmd = f"cd .. && python mp_run.py --sim eif_attr_stim --path {os.path.join(base_path, 'cli')} --simtime {simtime} --perturbation [0.0:0.205:0.1] --norm {norm} --weighted {weighted}"

In [ ]:
# res, err = utils.run_cmd(cmd)
# print(res)
# if len(err) > 0:
#     print(err)

## analyzing data

In [ ]:
t_start = 200.0
t_end = None

### general analysis

In [ ]:
def analyze(path, t_start, t_end):
    with persistence.FileMap(path, mode="read") as data:
        analyzer = ExperimentAnalysis.ExperimentAnalysis(
            experiment_data=data, t_start=t_start, t_end=t_end
        )
        # oscillation
        analyzer.analyze_instantaneous_rate()
        analyzer.analyze_smoothed_rate(window_size=1.0)
        analyzer.analyze_peaks()
        analyzer.analyze_cell_rate()
        analyzer.analyze_power_spectral_density(separate_intervals=True)
        analyzer.analyze_synchronization_frequency()
        # analyzer.analyze_total_synaptic_conductance(pop_e="E", pop_i="I")

        # attractor

        analyzer.analyze_snapshots(pop_name="E")
        analyzer.analyze_similarity_distribution(pop_name="E")

        sparsity = data["persist_data"]["E"]["sparsity"]
        pattern = data["persist_data"]["E"]["pattern"]

        analysis = analyzer.report

        dt = data["meta"]["dt"]["value"] * 1000
        t = data["meta"]["t"]["value"] * 1000
        t_start, t_end, t_last = utils.compute_time_interval(t, dt, t_start, t_end)

        stimulus_pattern = data["persist_data"]["stimulus_block_interval"][
            "stimulus_pattern"
        ]
        # convert to ms
        stimulus_block_interval = data["persist_data"]["stimulus_block_interval"][
            "interval"
        ]
        # treat homogeneous and inhomogeneous drive
        if "value" in stimulus_block_interval.keys():
            # homogeneous case
            stimulus_block_interval = stimulus_block_interval["value"] * 1000
        else:
            # inhomogeneous case load dictionary
            stimulus_block_interval = stimulus_block_interval.load()

        spike_train_e = data["SpikeDeviceGroup"]["E"]["spike"]["spike_train"]["value"].load()
        spike_train_i = data["SpikeDeviceGroup"]["I"]["spike"]["spike_train"]["value"].load()


        params = dict(mp.file_name_parser(fname))


    sm_rate_e = analysis["SpikeDeviceGroup"]["E"]["smoothed_rate"]["value"]
    sm_rate_i = analysis["SpikeDeviceGroup"]["I"]["smoothed_rate"]["value"]

    troughs = analysis["SpikeDeviceGroup"]["E"]["peaks"]["mins"]
    peaks = analysis["SpikeDeviceGroup"]["E"]["peaks"]["maxs"]

    sync_freq = analysis["SpikeDeviceGroup"]["E"]["synchronization_frequency"][
        "frequency"
    ]["value"]


    snapshots = analysis["SpikeDeviceGroup"]["E"]["snapshots"]
    similarity_distribution = analysis["SpikeDeviceGroup"]["E"]["pattern"][
        "similarity_distribution"
    ]

    return t_start, t_end, dt, peaks, troughs, snapshots, similarity_distribution, stimulus_pattern, stimulus_block_interval, pattern, sync_freq, sm_rate_e, sm_rate_i, spike_train_e, spike_train_i, params, analysis


In [ ]:
t_start, t_end, dt, peaks, troughs, snapshots, similarity_distribution, stimulus_pattern, stimulus_block_interval, pattern, sync_freq, pop_rate_e, pop_rate_i, spike_train_e, spike_train_i, params, analysis_data = analyze(os.path.join(base_path_code, fname), t_start, t_end)
t_startu, t_endu, dtu, peaksu, troughsu, snapshotsu, similarity_distributionu, stimulus_patternu, stimulus_block_intervalu, patternu, sync_frequ, pop_rate_eu, pop_rate_iu, spike_train_eu, spike_train_iu, paramsu, analysis_datau = analyze(os.path.join(base_path_code, fnameu), t_start, t_end)

### analysis of pvalues

In [ ]:
# compute pvalues for the stimulus pattern

def compute_pvals(snapshots,similarity_distribution, stimulus_pattern, stimuluspatternidx):
    # snapshots (C x pattern_length) where C num snapshots
    pattern_length = snapshots.shape[1]
    num_patterns = similarity_distribution.shape[0]
    num_spikes_per_snapshot = np.sum(snapshots, axis=1)
            
    # print(similarity_distribution.shape)
    similarity = similarity_distribution[stimuluspatternidx].ravel()


    number_ones = np.sum(stimulus_pattern)
    pattern_length = stimulus_pattern.size

    pvalues = np.zeros_like(similarity, dtype=float)
    for i in tqdm.tqdm(range(similarity.size)):
        p = num_spikes_per_snapshot[i] / pattern_length
        pval = attractor.pvalue_snapshot(pattern_length, number_ones, similarity[i], p)
        pvalues[i] = pval
    
    return pvalues

In [ ]:
# x = np.zeros_like(np.arange(20))
# x[3] = 2
# print(x)

In [ ]:
pvals = compute_pvals(snapshots, similarity_distribution, stimulus_pattern, stimuluspatternidx)
pvalsu = compute_pvals(snapshotsu, similarity_distributionu, stimulus_pattern, stimuluspatternidx)

## plot

In [ ]:
import plot
import matplotlib.pyplot as plt

### plotting oscillation dynamics

In [ ]:
def plot_osc(path, analysis_data, t_start):
    with persistence.FileMap(path, mode="read") as data:
        plotter = plot.ExperimentPlotter(data=data, analysis=analysis_data, figsize=(20,10), t_start=t_start, pop_name_e="E", pop_name_i="I")

        plotter.plot_spike_train()
        # plotter.plot_voltage()
        plotter.plot_instantaneous_rate(pop_name=["E", "I"])
        plotter.plot_smoothed_rate(pop_name=["E"])
        plotter.plot_smoothed_rate(pop_name=["I"])

        plotter.draw()

        plotter.show()


        plotter = plot.ExperimentPlotter(data=data, analysis=analysis_data, figsize=(20,10), sharex=True, t_start=100, pop_name_e="E", pop_name_i="I")
        plotter.plot_power_spectrum(pop_name="E")
        plotter.plot_power_spectrum(pop_name="I")

        plotter.draw()
        plotter.show()



        plotter = plot.ExperimentPlotter(data=data, analysis=analysis_data, figsize=(20,10), sharex=True, t_start=100, pop_name_e="E", pop_name_i="I")
        plotter.plot_power_spectogram_over_time(pop_name="E")
        plotter.plot_power_spectogram_over_time(pop_name="I")
        plotter.draw()
        plotter.show()

        plotter = plot.ExperimentPlotter(data=data, analysis=analysis_data, figsize=(20,10), sharex=True, t_start=100, pop_name_e="E", pop_name_i="I")
        plotter.plot_cell_rate(pop_name="E")
        plotter.plot_cell_rate(pop_name="I")
        plotter.draw()
        plotter.show()

In [ ]:
print(f"Oscillation plot {fname}")
plot_osc(os.path.join(base_path_code, fname), analysis_data, t_start)
print(f"Oscillation plot {fnameu}")
plot_osc(os.path.join(base_path_code, fname), analysis_datau, t_start)

### plot attractor dynamics

In [ ]:

def base_name_from_path(path: str):
    return os.path.split(path)[1]

# parameters with u are simply parameters corresponding to the unweighted sim
def plot_similarity_blocked_stimulus_sliding_window_delta(
    t_start, 
    dt,
    t_end,
    pattern, 
    stimulus_pattern,
    stimulus_block_interval,
    similarity_distribution,
    similarity_distributionu,
    pvals,
    pvalsu,
    params, 
    peaks, 
    peaksu,
    troughs, 
    troughsu,
    sync_freq, 

    
):
    """
    :param pvals: pvalues (C,) weighted pvalues for stimulus_pattern
    :param pvalsu: pvalues (C,) unweighted pvalues for stimulus_pattern 
    :param peaks: indices of the peaks at simulation time step dt resolution
    :param troughs: indices of the troughs at simulation time step dt resolution
    """

    time = np.arange(t_start, t_end + dt, dt)

    stimulus_onset = stimulus_block_interval[:, 0]

    stimulus_length = stimulus_block_interval[:, 1] - stimulus_block_interval[:, 0]

    stimulus_length = stimulus_length[0]

    #time = np.arange(t_start, t_end + dt, dt)


    # parameters
    significance = 0.05
    window_length = 20.0  # stimulus_length
    # resolution at 0.1 ms
    window_step = 0.1
    first_stim_onset = stimulus_onset[0]
    if stimulus_onset.size < 2 or not np.allclose(
        stimulus_onset[1:] - stimulus_onset[0:-1], stimulus_onset[1] - stimulus_onset[0]
    ):
        raise ValueError(
            "a constant stimulus_onset_interval required between onsets of the stimulus and"
            + " at least two stimulus presentations. "
            + f"Length of stimulus_presentations {stimulus_onset.size} and stimulus_onset_intervals {stimulus_onset[1:] - stimulus_onset[0:-1]}."
        )

    inter_onset_interval = stimulus_onset[1] - stimulus_onset[0]


    # as only pvalues for stimulus pattern are computed here - just reuse for all patterns and ignore
    pvals_expanded = np.tile(pvals, pattern.shape[0]).reshape(pattern.shape[0],-1)
    pvalsu_expanded = np.tile(pvalsu, pattern.shape[0]).reshape(pattern.shape[0],-1)

    assert np.all(pvals_expanded[0] == pvals)
    # weighted snaps



    indices = attractor.indices_snapshots_blocked_stimulus_sliding_window(
        pvals_expanded,
        stimulus_pattern,
        pattern,
        time[troughs],
        time[peaks],
        first_stim_onset,
        t_end,
        inter_onset_interval,
        window_length,
        window_step,
    )
    pvalue = [np.hstack([pvals_expanded[ix] for ix in idx]) for idx in indices]

    #print([p.shape for p in pvalue])

    sims = [np.hstack([similarity_distribution[ix] for ix in idx]) for idx in indices]

    # use sims now that we have them
    (sign_snaps, snap_pvals) = tuple(
        zip(
            *[
                (np.sum(pv <= significance) / pv.size, pv)
                if pv.size > 0
                else (0.0, np.ndarray([]))
                for pv in pvalue
            ]
        )
    )

    
    # print("sign_snaps:\n",sign_snaps)
    # print("snap_pvals:\n",snap_pvals)
    mean_pvals = np.array([np.mean(sp) for sp in snap_pvals])
    sign_snaps = np.array(sign_snaps)
    mean_sims = np.array([np.mean(sm) for sm in sims])

    # unwieghted snaps
    indices = attractor.indices_snapshots_blocked_stimulus_sliding_window(
        pvalsu_expanded,
        stimulus_pattern,
        pattern,
        time[troughsu],
        time[peaksu],
        first_stim_onset,
        t_end,
        inter_onset_interval,
        window_length,
        window_step,
    )
    pvalue_unweighted = [
        np.hstack([pvalsu_expanded[ix] for ix in idx]) for idx in indices
    ]

    # use sims now that we have them
    (sign_snaps_unweighted, snap_pvals_unweighted) = tuple(
        zip(
            *[
                (np.sum(pv <= significance) / pv.size, pv)
                if pv.size > 0
                else (0.0, np.ndarray([]))
                for pv in pvalue_unweighted
            ]
        )
    )
    sims_unweighted = [
        np.hstack([similarity_distributionu[ix] for ix in idx])
        for idx in indices
    ]
    mean_pvals_unweighted = np.array([np.mean(sp) for sp in snap_pvals_unweighted])
    sign_snaps_unweighted = np.array(sign_snaps_unweighted)
    mean_sims_unweighted = np.array([np.mean(sm) for sm in sims_unweighted])
    fig, ax = plt.subplots(2, 1, figsize=(10, 10))

    ttl = " | ".join([f"{p}: {v}" for p, v in params.items()])

    pop_name = "E"

    # one exemplary stimulus_onset , eg the second one - first pot not suitable as t_start may coincide with onset, for plotting
    example_onset = stimulus_onset[1]
    for i, zoomed in enumerate([False, True]):
        plot.plot_similarity_blocked_stimulus_sliding_window_delta(
            fig,
            ax[i],
            example_onset,
            inter_onset_interval,
            stimulus_length,
            window_step,
            sign_snaps=sign_snaps,
            mean_pvals=mean_pvals,
            mean_similarity=mean_sims,
            sign_snaps_unweighted=sign_snaps_unweighted,
            mean_pvals_unweighted=mean_pvals_unweighted,
            mean_similarity_unweighted=mean_sims_unweighted,
            zoomed=zoomed,
        )
    ax[0].set_title(
        f"similarity per pattern {pop_name} Sync Freq. {sync_freq:.2f} {ttl}",
        wrap=True,
    )
    return fig




In [ ]:

plot_similarity_blocked_stimulus_sliding_window_delta(
    t_start, 
    dt,
    t_end,
    pattern, 
    stimulus_pattern,
    stimulus_block_interval,
    similarity_distribution,
    similarity_distributionu,
    pvals,
    pvalsu,
    params, 
    peaks, 
    peaksu,
    troughs, 
    troughsu,
    sync_freq, 
)

#### plot psth

In [ ]:
def plot_psth(t_start, t_end, dt, stimulus_block_interval, stimulus_pattern, troughs, peaks, spike_train):

    time = np.arange(t_start, t_end + dt, dt)

    # spike train
    num_spikes = sum([v.size for v in spike_train.values()])
    spike_train_ids = np.zeros(num_spikes, dtype=int)
    spike_train_times = np.zeros(num_spikes, dtype=float)
    offset = 0
    for neuron, train in spike_train.items():
        spike_train_ids[offset : offset + train.size] = int(neuron)
        # in [ms]
        spike_train_times[offset : offset + train.size] = train * 1000

        offset += train.size

    (
        stimulus_onset,
        stimulus_end,
        stimulus_length,
        inter_presentation_interval,
    ) = analysis.compute_stimulus_characteristics(stimulus_block_interval)

    # curtail troughs, peaks and time to complete presentation cycles
    troughs = troughs[time[troughs] <= stimulus_end[-1]]
    peaks = peaks[time[peaks] <= stimulus_end[-1]]
    time = time[time <= stimulus_end[-1]]

    # separate snapshots into troughs, peaks, stimulus
    (
        trough_cycle_idx,
        peak_cycle_idx,
    ) = attractor.separate_presentation_cycles(
        time[troughs],
        time[peaks],
        stimulus_onset,
    )

    # presentation cycle times
    first_stim_onset = stimulus_onset[0]
    pres_beg, pres_end = attractor.resolve_time_interval(
        stimulus_onset, inter_presentation_interval, stimulus_length
    )

    # troughs
    # ith entry ~ trough_cycle_idx[i] and encompasses spikes relevant for stimulus_onset[i]
    spike_train_trough_idx = [
        np.logical_and(
            spike_train_times >= pres_beg[idx],
            spike_train_times < pres_end[idx],
        )
        for idx in trough_cycle_idx
    ]
    resolved_spike_times_trough = [
        spike_train_times[idx] - stimulus_onset[trough_cycle_idx[i]]
        for i, idx in enumerate(spike_train_trough_idx)
    ]
    spike_train_ids_trough = [
        spike_train_ids[idx] for idx in spike_train_trough_idx
    ]

    # peaks
    spike_train_peak_idx = [
        np.logical_and(
            spike_train_times >= pres_beg[idx],
            spike_train_times < pres_end[idx],
        )
        for idx in np.nonzero(peak_cycle_idx)[0]
    ]
    resolved_spike_times_peak = [
        spike_train_times[idx] - stimulus_onset[peak_cycle_idx[i]]
        for i, idx in enumerate(spike_train_peak_idx)
    ]
    spike_train_ids_peak = [
        spike_train_ids[idx] for idx in spike_train_peak_idx
    ]

    fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
    plt.subplots_adjust(wspace=0, hspace=0)

    for i, (spike_time, spike_id, num_presentations) in enumerate(
        zip(
            [resolved_spike_times_trough, resolved_spike_times_peak],
            [spike_train_ids_trough, spike_train_ids_peak],
            [len(spike_train_trough_idx), len(spike_train_peak_idx)],
        )
    ):
        # print(type(spike_time))
        if len(spike_time) > 0:
            plot.psth(
                fig,
                ax[i],
                np.hstack(spike_time),
                np.hstack(spike_id),
                stimulus_pattern,
                num_presentations,
                inter_presentation_interval,
                stimulus_length,
                num_bins=48,
            )

    ax[0].set_title(
        f"psth for presentation cycles trough (top, # cycles {trough_cycle_idx.size}) and peak (bottom, # cycles {peak_cycle_idx.size})"
    )


In [ ]:
with persistence.FileMap(os.path.join(base_path_code, fname)) as fl:
    spike_train = fl["SpikeDeviceGroup"]["E"]["spike"]["spike_train"][
        "value"
    ].load()

with persistence.FileMap(os.path.join(base_path_code, fnameu)) as fl:
    spike_trainu = fl["SpikeDeviceGroup"]["E"]["spike"]["spike_train"][
        "value"
    ].load()

In [ ]:
print(fname)
plot_psth(t_start, t_end, dt, stimulus_block_interval, stimulus_pattern, troughs, peaks, spike_train)
print(fnameu)
plot_psth(t_start, t_end, dt, stimulus_block_interval, stimulus_pattern, troughsu, peaksu, spike_trainu)

### spike presentation cycles

In [ ]:

def plot_spike_presentation_cycle(t_start, t_end, dt, spike_train_e, spike_train_i, pop_rate_e, pop_rate_i, stimulus_block_interval, stimulus_pattern, peaks, troughs):
    fig, ax = plt.subplots(4, 1, figsize=(20, 10))
    plt.subplots_adjust(hspace=0.35)

    time = np.arange(t_start, t_end + dt, dt)

    for pop_i, (spike_train, pop_rate, pn) in enumerate(
        zip(
            [spike_train_e, spike_train_i],
            [pop_rate_e, pop_rate_i],
            ["E", "I"],
        )
    ):

        num_spikes = sum([v.size for v in spike_train.values()])
        spike_train_ids = np.zeros(num_spikes, dtype=int)
        spike_train_times = np.zeros(num_spikes, dtype=float)
        offset = 0
        for neuron, train in spike_train.items():
            spike_train_ids[offset : offset + train.size] = int(neuron)
            # in [ms]
            spike_train_times[offset : offset + train.size] = train * 1000

            offset += train.size

        (
            stimulus_onset,
            stimulus_end,
            stimulus_length,
            inter_presentation_interval,
        ) = analysis.compute_stimulus_characteristics(
            stimulus_block_interval
        )

        # separate snapshots into troughs, peaks, stimulus
        (
            trough_cycle_idx,
            peak_cycle_idx,
        ) = attractor.separate_presentation_cycles(
            time[troughs],
            time[peaks],
            stimulus_onset,
        )

        cycles = [trough_cycle_idx, peak_cycle_idx]
        for i, idx in enumerate(cycles):
                            if idx.size > 0:
                                plot.plot_spike_train_presentation_cycle(
                                    fig,
                                    ax[pop_i * len(cycles) + i],
                                    spike_train_ids,
                                    spike_train_times,
                                    stimulus_onset[idx][1 if idx.size > 1 else 0],
                                    inter_presentation_interval,
                                    stimulus_length,
                                    pop_rate,
                                    t_start,
                                    t_end,
                                    dt,
                                    stimulus_pattern=stimulus_pattern
                                    if pn == "E"
                                    else None,
                                )

    ax[0].set_title(
        f"spikes during a presentation cycle: E group trough, E group peak, I group trough, I group peak (top to bottom)"
    )

In [ ]:
print(fname)
plot_spike_presentation_cycle(t_start, t_end, dt, spike_train_e, spike_train_i, pop_rate_e, pop_rate_i, stimulus_block_interval, stimulus_pattern, peaks, troughs)
print(fnameu)
plot_spike_presentation_cycle(t_start, t_end, dt, spike_train_eu, spike_train_iu, pop_rate_eu, pop_rate_iu, stimulus_block_interval, stimulus_pattern, peaksu, troughsu)

### scale

In [ ]:
with persistence.FileMap(os.path.join(base_path_code, fname)) as data:
    scale = data["Synapse"]["S_E_E"]["synapse_params"]["scale"]


In [ ]:
def plot_scale(scale):
    vals, counts = np.unique(scale, return_counts=True)

    fig, ax = plt.subplots(1, 1)
    ax.scatter(vals, np.log(counts))
    ax.set_xlabel("synaptic input scale")
    ax.set_ylabel("log count")
    ax.set_title("Synaptic input scale matrix")

In [ ]:
plot_scale(scale)